In [ ]:
devtools::install_github("clairbarnes/rwwa")
library(rwwa)

ERROR: Error in loadNamespace(i, c(lib.loc, .libPaths()), versionCheck = vI[[i]]): namespace ‘fastmap’ 1.1.0 is already loaded, but >= 1.1.1 is required


## Per grid cell

In [27]:
nino <- read.csv("nino_djf.csv", col.names = c("year", "nino"))
nino$year <- nino$year - 1

gmst <- read.table("gmst.txt", col.names = c("year", "gmst"))
gmst$gmst <- gmst$gmst - gmst$gmst[gmst$year == 2023]

df <- read.csv("era5_flattened.csv", skip = 3, header = F)
colnames(df)[1] <- "year"

df <- merge(merge(nino, gmst), df)

In [32]:
cov_2023 <- df[df$year == 2023,c("gmst", "nino")]
cov_cf <- rbind("hist" = cov_2023 - c(1.2, 0),
                "neut" = c(cov_2023$gmst, 0))

In [34]:
# fit nonstationary model to all grid cells
models <- sapply(colnames(df)[-(1:3)], function(cnm) {
    df_c <- df[,c("year", "gmst", "nino", cnm)]
    fit_ns("gev", "fixeddisp", df_c, cnm, c("gmst", "nino"), lower = F)
}, simplify = F)

In [35]:
# get all model resuls & save as .csv to be reconstructed & plotted in python
fitted_res <- sapply(models, mdl_ests, cov_f = cov_2023, cov_cf = cov_cf)
write.csv(fitted_res, paste0("gridded-res_era5.csv"))